#Load Data

In [ ]:
'''
!pip install matplotlib scipy opencv-python pandas glob torch
!apt-get update
!apt-get install unzip
!apt-get install ffmpeg libsm6 libxext6  -y
!wget 'https://www.dropbox.com/s/l0y7cqqymjef2sh/tiny-imagenet-200.zip?dl=0'
!mv 'tiny-imagenet-200.zip?dl=0' 'ImageNetData.zip'
!unzip 'ImageNetData.zip'
!ls
''

In [ ]:
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class ImageNet(Dataset):
    def __init__(self):
        self.imgs_path = "imagenet/"
        sub_folders = glob.glob(self.imgs_path + "*")
        self.data = []
        self.classes = []
        for sub_folder in sub_folders:
            file_list = glob.glob(sub_folder + "/*")
            for class_path in file_list:
                class_name = class_path.split("/")[-1]
                for img_path in glob.glob(class_path + "/*.JPEG"):
                    self.data.append([img_path, class_name])
                    self.classes.append(class_name)
        self.classes = list(set(self.classes))
        self.class_map = dict()
        for i in range(len(self.classes)):
            self.class_map[self.classes[i]] = i
        self.img_dim = (32, 32)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img).float()
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor(class_id)
        return img_tensor, class_id

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision

from torchvision import *
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import time
import copy
import os

from models import *
from utils import progress_bar
from utils import EarlyStopping

batch_size = 32

transforms = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Resize([32,32])
])

dataset = ImageNet()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

images, labels = next(iter(train_dataloader)) 
print("images-size:", images.shape)

out = torchvision.utils.make_grid(images)
print("out-size:", labels.shape)


# Training

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(train_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return train_loss


# Testing

In [ ]:
def test(epoch,model_name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            inputs = inputs.permute(0,3,1,2)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(test_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/'+model_name+'.pth')
        best_acc = acc
    return test_loss


In [ ]:
classes = dataset.classes
lr = 0.01
n_epochs = 20

In [ ]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DenseNet121(num_classes=len(classes))
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DenseNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1